In [11]:
#!/usr/bin/env python3
from argparse import ArgumentParser, FileType
from importlib import import_module
from itertools import count
import os

import h5py
import json
import numpy as np
from sklearn.metrics import average_precision_score
import tensorflow as tf

import common
import loss
from matplotlib import pyplot as plt

In [12]:
class Args:
    excluder='market1501'
    query_dataset='/home/k/kajal/triplet-reid/data/market1501/market1501_query.csv'
    query_embeddings='/home/k/kajal/triplet-reid/experiments/mkt_recon_ce_exp/query_embeddings.h5'
    gallery_dataset='/home/k/kajal/triplet-reid/data/market1501/market1501_test.csv'
    gallery_embeddings='/home/k/kajal/triplet-reid/experiments/mkt_recon_ce_exp/gallery_embeddings.h5'
    metric='euclidean'
    #filename='None'
    batch_size=18

In [13]:
args=Args()
args.gpu=0

In [14]:
# Verify that parameters are set correctly.
#args = parser.parse_args()

# Load the query and gallery data from the CSV files.
query_pids, query_fids = common.load_dataset(args.query_dataset, None)
gallery_pids, gallery_fids = common.load_dataset(args.gallery_dataset, None)
# Load the two datasets fully into memory.
with h5py.File(args.query_embeddings, 'r') as f_query:
    query_embs = np.array(f_query['emb'])
with h5py.File(args.gallery_embeddings, 'r') as f_gallery:
    gallery_embs = np.array(f_gallery['emb'])

# Just a quick sanity check that both have the same embedding dimension!
query_dim = query_embs.shape[1]
gallery_dim = gallery_embs.shape[1]
if query_dim != gallery_dim:
    raise ValueError('Shape mismatch between query ({}) and gallery ({}) '
                     'dimension'.format(query_dim, gallery_dim))

# Setup the dataset specific matching function
excluder = import_module('excluders.' + args.excluder).Excluder(gallery_fids)

In [24]:
batch_pids, batch_fids, batch_embs = tf.data.Dataset.from_tensor_slices((query_pids, query_fids, query_embs))#.batch(args.batch_size)

ValueError: too many values to unpack (expected 3)

In [20]:
def main():
    # Verify that parameters are set correctly.
    
    # We go through the queries in batches, but we always need the whole gallery
    batch_pids, batch_fids, batch_embs = tf.data.Dataset.from_tensor_slices((query_pids, query_fids, query_embs)
    ).batch(args.batch_size)#.make_one_shot_iterator().get_next()
    iterator = iter(batch_pids,batch_fids,batch_embs)
    next_element = iterator.get_next()
    #tf.compat.v1.data.make_one_shot_iterator
    batch_distances = loss.cdist(batch_embs, gallery_embs, metric=args.metric)
    id_batch_distances = loss.cdist(batch_embs, gallery_embs, 'ce')


    # Loop over the query embeddings and compute their APs and the CMC curve.
    aps = []
    cmc = np.zeros(len(gallery_pids), dtype=np.int32)
    acc = np.zeros(len(gallery_pids), dtype=np.int32)

    with tf.Session() as sess:
        for start_idx in count(step=args.batch_size):
            try:
                # Compute distance to all gallery embeddings
                distances, dist, pids, fids = sess.run([
                    batch_distances, id_batch_distances, batch_pids, batch_fids])
                print('\rEvaluating batch {}-{}/{}'.format(
                        start_idx, start_idx + len(fids), len(query_fids)),
                      flush=True, end='')
            except tf.errors.OutOfRangeError:
                print()  # Done!
                break

            # Convert the array of objects back to array of strings
            #print(pids)
            pids, fids = np.array(pids, '|U'), np.array(fids, '|U')
            print(pids)

            # Compute the pid matches
            pid_matches = gallery_pids[None] == pids[:,None]

            # Get a mask indicating True for those gallery entries that should
            # be ignored for whatever reason (same camera, junk, ...) and
            # exclude those in a way that doesn't affect CMC and mAP.
            mask = excluder(fids)
            distances[mask] = np.inf
            pid_matches[mask] = False

            # Keep track of statistics. Invert distances to scores using any
            # arbitrary inversion, as long as it's monotonic and well-behaved,
            # it won't change anything.
            scores = 1 / (1 + distances)
            for i in range(len(distances)):
                ap = average_precision_score(pid_matches[i], scores[i])

                if np.isnan(ap):
                    print()
                    print("WARNING: encountered an AP of NaN!")
                    print("This usually means a person only appears once.")
                    print("In this case, it's because of {}.".format(fids[i]))
                    print("I'm excluding this person from eval and carrying on.")
                    print()
                    continue

                aps.append(ap)
                # Find the first true match and increment the cmc data from there on.
                k = np.where(pid_matches[i, np.argsort(distances[i])])[0][0]
                cmc[k:] += 1

                for j in range(len(dist)):
                    ids = np.where(pid_matches[j, np.argsort(dist[j])])[0][0]
                    acc[ids:] += 1

    # Compute the actual cmc and mAP values
    cmc = cmc / len(query_pids)
    acc = acc / len(query_pids)
    mean_ap = np.mean(aps)

    # Save important data
    if args.filename is not None:
        json.dump({'mAP': mean_ap, 'CMC': list(cmc), 'aps': list(aps)}, args.filename)

    # Print out a short summary.
    print('mAP: {:.2%} | top-1: {:.2%} top-2: {:.2%} | top-5: {:.2%} | top-10: {:.2%} | acc: {:.2%}'.format(
        mean_ap, cmc[0], cmc[1], cmc[4], cmc[9], acc[0]))

if __name__ == '__main__':
    main()

ValueError: too many values to unpack (expected 3)

In [15]:
# We go through the queries in batches, but we always need the whole gallery
batch_pids, batch_fids, batch_embs = tf.data.Dataset.from_tensor_slices(
    (query_pids, query_fids, query_embs)
).batch(args.batch_size).make_one_shot_iterator().get_next()
batch_distances = loss.cdist(batch_embs, gallery_embs, metric=args.metric)

AttributeError: 'BatchDataset' object has no attribute 'make_one_shot_iterator'

In [9]:


# Loop over the query embeddings and compute their APs and the CMC curve.
aps = []
cmc = np.zeros(len(gallery_pids), dtype=np.int32)
with tf.Session() as sess:
    for start_idx in count(step=args.batch_size):
        try:
            # Compute distance to all gallery embeddings
            distances, pids, fids = sess.run([
                batch_distances, batch_pids, batch_fids])
            print('\rEvaluating batch {}-{}/{}'.format(
                    start_idx, start_idx + len(fids), len(query_fids)),
                  flush=True, end='')
        except tf.errors.OutOfRangeError:
            print()  # Done!
            break

        # Convert the array of objects back to array of strings
        pids, fids = np.array(pids, '|U'), np.array(fids, '|U')
        #print(fids)
        #plt.imshow(fids)
        # Compute the pid matches
        pid_matches = gallery_pids[None] == pids[:,None]
        # Get a mask indicating True for those gallery entries that should
        # be ignored for whatever reason (same camera, junk, ...) and
        # exclude those in a way that doesn't affect CMC and mAP.
        mask = excluder(fids)
        distances[mask] = np.inf
        pid_matches[mask] = False

        # Keep track of statistics. Invert distances to scores using any
        # arbitrary inversion, as long as it's monotonic and well-behaved,
        # it won't change anything.
        scores = 1 / (1 + distances)
        for i in range(len(distances)):
            ap = average_precision_score(pid_matches[i], scores[i])

            if np.isnan(ap):
                print()
                print("WARNING: encountered an AP of NaN!")
                print("This usually means a person only appears once.")
                print("In this case, it's because of {}.".format(fids[i]))
                print("I'm excluding this person from eval and carrying on.")
                print()
                continue

            aps.append(ap)
            print(aps)
            # Find the first true match and increment the cmc data from there on.
            k = np.where(pid_matches[i, np.argsort(distances[i])])[0][0]
            cmc[k:] += 1

# Compute the actual cmc and mAP values
cmc = cmc / len(query_pids)
print()
mean_ap = np.mean(aps)

# Save important data
#if args.filename is not None:
    #json.dump({'mAP': mean_ap, 'CMC': list(cmc), 'aps': list(aps)}, args.filename)

# Print out a short summary.
print('mAP: {:.2%} | top-1: {:.2%} top-2: {:.2%} | top-5: {:.2%} | top-10: {:.2%}'.format(
    mean_ap, cmc[0], cmc[1], cmc[4], cmc[9]))


AttributeError: 'BatchDataset' object has no attribute 'make_one_shot_iterator'

In [ ]:

query_path, _ = image_datasets['query'].imgs[i]
query_label = query_label[i]
print(query_path)
print('Top 10 images are as follow:')
try: # Visualize Ranking Result 
    # Graphical User Interface is needed
    fig = plt.figure(figsize=(16,4))
    ax = plt.subplot(1,11,1)
    ax.axis('off')
    imshow(query_path,'query')
    for i in range(10):
        ax = plt.subplot(1,11,i+2)
        ax.axis('off')
        img_path, _ = image_datasets['gallery'].imgs[index[i]]
        label = gallery_label[index[i]]
        imshow(img_path)
        if label == query_label:
            ax.set_title('%d'%(i+1), color='green')
        else:
            ax.set_title('%d'%(i+1), color='red')
        print(img_path)
except RuntimeError:
    for i in range(10):
        img_path = image_datasets.imgs[index[i]]
        print(img_path[0])
    print('If you want to see the visualization of the ranking result, graphical user interface is needed.')

fig.savefig("show.png")

In [ ]:
query_pids